# Construcción de Operaciones

Para ejecutar todos los ejemplos se debe importar la librería. Se sugiere utilizar siempre el alias `qcf`. 

In [1]:
import qcfinancial as qcf
import pandas as pd

Se verifica la versión y build de `qcfinancial`.

In [2]:
qcf.id()

'version: 0.10.0, build: 2024-06-09 08:01'

El siguiente diccionario se utiliza para dar formato a las columnas de los `pandas.DataFrames` que se utilizarán.

In [54]:
format_dict = {
    'nominal': '{0:,.2f}', 
    'nocional': '{0:,.2f}', 
    'amort': '{0:,.2f}', 
    'amortizacion': '{0:,.2f}', 
    'interes': '{0:,.2f}',
    'monto': '{0:,.2f}',
    'flujo': '{0:,.2f}',
    'flujo_moneda_pago': '{0:,.2f}',
    'flujo_en_clp': '{0:,.2f}',
    'icp_inicial': '{0:,.2f}', 
    'icp_final': '{0:,.2f}',
    'uf_inicial': '{0:,.2f}', 
    'uf_final': '{0:,.2f}',
    'valor_tasa': '{0:,.4%}', 
    'spread': '{0:,.4%}', 
    'gearing': '{0:,.2f}',
    'amortizacion_moneda_pago': '{0:,.2f}', 
    'interes_moneda_pago': '{0:,.2f}', 
    'valor_indice_fx': '{0:,.2f}'
}

## Legs

Los objetos de tipo `Leg` son una lista (o vector) de objetos `Cashflow` y representan una pata de un instrumento financiero. un objeto de tipo `Leg` puede construirse *a mano* es decir, dando de alta cashflows y agregándolos uno a uno o con algunos métodos de conveniencia cuyo funcionamiento se mostrará.

### Construcción Manual

Se verá como construir un `Leg` con 2 `SimpleCashflow` de forma *manual*. En particular, este objeto `Leg` podría representar una operación FX por entrega física.

In [4]:
leg = qcf.Leg()
fecha_vcto = qcf.QCDate(20, 9, 2018)
simple_cashflow_1 = qcf.SimpleCashflow(
    fecha_vcto,    # fecha del flujo
    1_000,         # monto
    qcf.QCUSD()    # moneda
) 

simple_cashflow_2 = qcf.SimpleCashflow(
    fecha_vcto,    # fecha del flujo
    -900_000,      # monto
    qcf.QCCLP())   # moneda

leg.append_cashflow(simple_cashflow_1)
leg.append_cashflow(simple_cashflow_2)

Se observa el resultado

In [5]:
data = [qcf.show(leg.get_cashflow_at(i)) for i in range(0, 2)]
df = pd.DataFrame(data, columns=qcf.get_column_names('SimpleCashflow', ''))
df.style.format(format_dict)

,fecha_pago,monto,moneda
0,2018-09-20,"1,000.00",USD
1,2018-09-20,"-900,000.00",CLP


## Construcción Asistida de un `FixedRateLeg`

Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateCashflow`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateCasflow` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

Se da de alta los parámetros requeridos

In [6]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(5, 11, 2019)
fecha_final = qcf.QCDate(5, 11, 2023)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
calendario.add_holiday(qcf.QCDate(31, 12, 2019))
lag_pago = 0
nominal = 100000.0
amort_es_flujo = False
tasa_cupon = qcf.QCInterestRate(.03, qcf.QCAct360(), qcf.QCLinearWf())
moneda = qcf.QCCLF()
es_bono = False

Se da de alta el objeto.

In [7]:
fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

Como en el capítulo anterior, se puede lograr una buena visualización mucho mejor del resultado utilizando un Dataframe de pandas y el método `show`.

In [8]:
# Se define un list donde almacenar los resultados de la función show
tabla = [qcf.show(fixed_rate_leg.get_cashflow_at(i)) for i in range(0, fixed_rate_leg.size())]
df = pd.DataFrame(tabla, columns=qcf.get_column_names('FixedRateCashflow', ''))

# Se despliega la data en este formato
df.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-11-05,2020-05-05,2020-05-05,"100,000.00",0.00,"1,516.67",False,"1,516.67",CLF,3.0000%,LinAct360
1,2020-05-05,2020-11-05,2020-11-05,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
2,2020-11-05,2021-05-05,2021-05-05,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
3,2021-05-05,2021-11-05,2021-11-05,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
4,2021-11-05,2022-05-05,2022-05-05,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
5,2022-05-05,2022-11-07,2022-11-07,"100,000.00",0.00,"1,550.00",False,"1,550.00",CLF,3.0000%,LinAct360
6,2022-11-07,2023-05-05,2023-05-05,"100,000.00",0.00,"1,491.67",False,"1,491.67",CLF,3.0000%,LinAct360
7,2023-05-05,2023-11-06,2023-11-06,"100,000.00","100,000.00","1,541.67",False,"1,541.67",CLF,3.0000%,LinAct360


### Otros `StubPeriod`

Período irregular corto al inicio (`qcf.StubPeriod.SHORTFRONT`)

In [9]:
fecha_inicio = qcf.QCDate(5, 11, 2019)
fecha_final = qcf.QCDate(31, 5, 2023)
periodo_irregular = qcf.StubPeriod.SHORTFRONT

fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

tabla = [qcf.show(fixed_rate_leg.get_cashflow_at(i)) for i in range(0, fixed_rate_leg.size())]
df = pd.DataFrame(tabla, columns=qcf.get_column_names('FixedRateCashflow', ''))
df.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-11-05,2019-12-30,2019-12-30,"100,000.00",0.00,458.33,False,458.33,CLF,3.0000%,LinAct360
1,2019-12-30,2020-06-30,2020-06-30,"100,000.00",0.00,"1,525.00",False,"1,525.00",CLF,3.0000%,LinAct360
2,2020-06-30,2020-12-31,2020-12-31,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
3,2020-12-31,2021-06-30,2021-06-30,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
4,2021-06-30,2021-12-31,2021-12-31,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
5,2021-12-31,2022-06-30,2022-06-30,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
6,2022-06-30,2022-12-30,2022-12-30,"100,000.00",0.00,"1,525.00",False,"1,525.00",CLF,3.0000%,LinAct360
7,2022-12-30,2023-06-30,2023-06-30,"100,000.00","100,000.00","1,516.67",False,"1,516.67",CLF,3.0000%,LinAct360


Período irregular corto al final (`qcf.StubPeriod.SHORTBACK`)

In [10]:
fecha_inicio = qcf.QCDate(5, 11, 2019)
fecha_final = qcf.QCDate(31, 5, 2023)
periodo_irregular = qcf.StubPeriod.SHORTBACK

fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

tabla = [qcf.show(fixed_rate_leg.get_cashflow_at(i)) for i in range(0, fixed_rate_leg.size())]
df = pd.DataFrame(tabla, columns=qcf.get_column_names('FixedRateCashflow', ''))
df.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-11-05,2020-05-05,2020-05-05,"100,000.00",0.00,"1,516.67",False,"1,516.67",CLF,3.0000%,LinAct360
1,2020-05-05,2020-11-05,2020-11-05,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
2,2020-11-05,2021-05-05,2021-05-05,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
3,2021-05-05,2021-11-05,2021-11-05,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
4,2021-11-05,2022-05-05,2022-05-05,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
5,2022-05-05,2022-11-07,2022-11-07,"100,000.00",0.00,"1,550.00",False,"1,550.00",CLF,3.0000%,LinAct360
6,2022-11-07,2023-05-05,2023-05-05,"100,000.00",0.00,"1,491.67",False,"1,491.67",CLF,3.0000%,LinAct360
7,2023-05-05,2023-05-31,2023-05-31,"100,000.00","100,000.00",216.67,False,216.67,CLF,3.0000%,LinAct360


Período irregular corto al final (`qcf.StubPeriod.LONGFRONT`)

In [11]:
fecha_inicio = qcf.QCDate(5, 11, 2019)
fecha_final = qcf.QCDate(31, 5, 2023)
periodo_irregular = qcf.StubPeriod.LONGFRONT

fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

tabla = [qcf.show(fixed_rate_leg.get_cashflow_at(i)) for i in range(0, fixed_rate_leg.size())]
df = pd.DataFrame(tabla, columns=qcf.get_column_names('FixedRateCashflow', ''))
df.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-11-05,2020-05-29,2020-05-29,"100,000.00",0.00,"1,716.67",False,"1,716.67",CLF,3.0000%,LinAct360
1,2020-05-29,2020-11-30,2020-11-30,"100,000.00",0.00,"1,541.67",False,"1,541.67",CLF,3.0000%,LinAct360
2,2020-11-30,2021-05-31,2021-05-31,"100,000.00",0.00,"1,516.67",False,"1,516.67",CLF,3.0000%,LinAct360
3,2021-05-31,2021-11-30,2021-11-30,"100,000.00",0.00,"1,525.00",False,"1,525.00",CLF,3.0000%,LinAct360
4,2021-11-30,2022-05-31,2022-05-31,"100,000.00",0.00,"1,516.67",False,"1,516.67",CLF,3.0000%,LinAct360
5,2022-05-31,2022-11-30,2022-11-30,"100,000.00",0.00,"1,525.00",False,"1,525.00",CLF,3.0000%,LinAct360
6,2022-11-30,2023-05-31,2023-05-31,"100,000.00","100,000.00","1,516.67",False,"1,516.67",CLF,3.0000%,LinAct360


Período irregular corto al final (`qcf.StubPeriod.LONGBACK`)

In [12]:
fecha_inicio = qcf.QCDate(5, 11, 2019)
fecha_final = qcf.QCDate(31, 5, 2023)
periodo_irregular = qcf.StubPeriod.LONGBACK

fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

tabla = [qcf.show(fixed_rate_leg.get_cashflow_at(i)) for i in range(0, fixed_rate_leg.size())]
df = pd.DataFrame(tabla, columns=qcf.get_column_names('FixedRateCashflow', ''))
df.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-11-05,2020-05-05,2020-05-05,"100,000.00",0.00,"1,516.67",False,"1,516.67",CLF,3.0000%,LinAct360
1,2020-05-05,2020-11-05,2020-11-05,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
2,2020-11-05,2021-05-05,2021-05-05,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
3,2021-05-05,2021-11-05,2021-11-05,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
4,2021-11-05,2022-05-05,2022-05-05,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
5,2022-05-05,2022-11-07,2022-11-07,"100,000.00",0.00,"1,550.00",False,"1,550.00",CLF,3.0000%,LinAct360
6,2022-11-07,2023-05-31,2023-05-31,"100,000.00","100,000.00","1,708.33",False,"1,708.33",CLF,3.0000%,LinAct360


Período irregular corto al final (`qcf.StubPeriod.LONGFRONT3`)

In [13]:
fecha_inicio = qcf.QCDate(5, 11, 2019)
fecha_final = qcf.QCDate(31, 5, 2023)
periodo_irregular = qcf.StubPeriod.LONGFRONT3 # Una de varias opciones parecidas para alargar el primer período de intereses

fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    moneda,
    es_bono
)

tabla = [qcf.show(fixed_rate_leg.get_cashflow_at(i)) for i in range(0, fixed_rate_leg.size())]
df = pd.DataFrame(tabla, columns=qcf.get_column_names('FixedRateCashflow', ''))
df.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2019-11-05,2020-12-31,2020-12-31,"100,000.00",0.00,"3,516.67",False,"3,516.67",CLF,3.0000%,LinAct360
1,2020-12-31,2021-06-30,2021-06-30,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
2,2021-06-30,2021-12-31,2021-12-31,"100,000.00",0.00,"1,533.33",False,"1,533.33",CLF,3.0000%,LinAct360
3,2021-12-31,2022-06-30,2022-06-30,"100,000.00",0.00,"1,508.33",False,"1,508.33",CLF,3.0000%,LinAct360
4,2022-06-30,2022-12-30,2022-12-30,"100,000.00",0.00,"1,525.00",False,"1,525.00",CLF,3.0000%,LinAct360
5,2022-12-30,2023-06-30,2023-06-30,"100,000.00","100,000.00","1,516.67",False,"1,516.67",CLF,3.0000%,LinAct360


## Construcción Asistida de un `CustomAmortFixedRateLeg`

En este ejemplo se construye un `Leg` con cashflows a tasa fija donde la estructura de amortizaciones es customizada.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `CustomNotionalAndAmort`: vector de capital vigente y amortizaciones customizado
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

Primero se da de alta los parámetros requeridos

In [14]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 1, 2028) 

calendario = qcf.BusinessCalendar(fecha_inicio, 20)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.NO
lag_pago = 0

valor_tasa = .03
tasa_cupon = qcf.QCInterestRate(
    valor_tasa, 
    qcf.QC30360(), 
    qcf.QCLinearWf()
)

moneda = qcf.QCCLF()
es_bono = False

Aquí se da de alta eñ vector de capitales vigentes y amortizaciones. Cada elemento del vector es el capital vigente y amortización del correspondiente cupón.

In [15]:
custom_notional_amort = qcf.CustomNotionalAmort()
custom_notional_amort.set_size(8)  # De la fecha inicio y fecha final se deduce que serán 8 cupones
for i in range(0, 8):
    custom_notional_amort.set_notional_amort_at(i, 1000.0 - i * 100.0, 100.0)
amort_es_flujo = False

Se da de alta el objeto.

In [16]:
fixed_rate_custom_leg = qcf.LegFactory.build_custom_amort_fixed_rate_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    custom_notional_amort,
    amort_es_flujo,
    tasa_cupon,
    moneda
)

In [17]:
tabla = [qcf.show(fixed_rate_custom_leg.get_cashflow_at(i)) for i in range(0, fixed_rate_custom_leg.size())]
df = pd.DataFrame(tabla, columns=qcf.get_column_names('FixedRateCashflow', ''))
df.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2024-01-31,2024-07-31,2024-07-31,"1,000.00",100.00,15.00,False,15.00,CLF,3.0000%,Lin30360
1,2024-07-31,2025-01-31,2025-01-31,900.00,100.00,13.50,False,13.50,CLF,3.0000%,Lin30360
2,2025-01-31,2025-07-31,2025-07-31,800.00,100.00,12.00,False,12.00,CLF,3.0000%,Lin30360
3,2025-07-31,2026-01-30,2026-01-30,700.00,100.00,10.50,False,10.50,CLF,3.0000%,Lin30360
4,2026-01-30,2026-07-31,2026-07-31,600.00,100.00,9.00,False,9.00,CLF,3.0000%,Lin30360
5,2026-07-31,2027-01-29,2027-01-29,500.00,100.00,7.46,False,7.46,CLF,3.0000%,Lin30360
6,2027-01-29,2027-07-30,2027-07-30,400.00,100.00,6.03,False,6.03,CLF,3.0000%,Lin30360
7,2027-07-30,2028-01-31,2028-01-31,300.00,100.00,4.50,False,4.50,CLF,3.0000%,Lin30360


## Función `leg_as_dataframe`

Para no seguir repitiendo el bloque de código anterior, se define la siguiente función.

In [18]:
def leg_as_dataframe(leg: qcf.Leg):
    """
    Envuelve un objeto qcf.Leg en un pd.DataFrame
    """
    if leg.size() == 0:
        raise ValueError("No cashflows")
    type_cashflows = leg.get_cashflow_at(0).get_type()
    tabla = [qcf.show(leg.get_cashflow_at(i)) for i in range(0, leg.size())]
    df = pd.DataFrame(tabla, columns=qcf.get_column_names(type_cashflows, ''))
    return df

Se testea con el ejemplo anterior.

In [19]:
leg_as_dataframe(fixed_rate_custom_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2024-01-31,2024-07-31,2024-07-31,"1,000.00",100.00,15.00,False,15.00,CLF,3.0000%,Lin30360
1,2024-07-31,2025-01-31,2025-01-31,900.00,100.00,13.50,False,13.50,CLF,3.0000%,Lin30360
2,2025-01-31,2025-07-31,2025-07-31,800.00,100.00,12.00,False,12.00,CLF,3.0000%,Lin30360
3,2025-07-31,2026-01-30,2026-01-30,700.00,100.00,10.50,False,10.50,CLF,3.0000%,Lin30360
4,2026-01-30,2026-07-31,2026-07-31,600.00,100.00,9.00,False,9.00,CLF,3.0000%,Lin30360
5,2026-07-31,2027-01-29,2027-01-29,500.00,100.00,7.46,False,7.46,CLF,3.0000%,Lin30360
6,2027-01-29,2027-07-30,2027-07-30,400.00,100.00,6.03,False,6.03,CLF,3.0000%,Lin30360
7,2027-07-30,2028-01-31,2028-01-31,300.00,100.00,4.50,False,4.50,CLF,3.0000%,Lin30360


## Construcción Asistida de un `FixedRateMultiCurrencyLeg`

Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateMultiCurrencyCashflow`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateMultiCurrencyCasflow` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal
- `QCCurrency`: moneda de los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago.
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámetros siguientes se puede visualizar el efecto de ellos en la construcción.

Primero se debe dar de alta un FXRateIndex

In [20]:
usd = qcf.QCUSD()
clp = qcf.QCCLP()
usdclp = qcf.FXRate(usd, clp)
one_d = qcf.Tenor('1D')
usdclp_obs = qcf.FXRateIndex(
    usdclp, 
    'USDOBS', 
    one_d, 
    one_d, 
    calendario
)

Luego se dan de alta los otros parámetros requeridos para la construcción

In [21]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 1969)
fecha_final = qcf.QCDate(31, 1, 1974) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.NO
lag_pago = 0
es_bono = False

Finalmente, se da de alta el objeto.

In [22]:
fixed_rate_mccy_leg = qcf.LegFactory.build_bullet_fixed_rate_mccy_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad,
    periodo_irregular,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    tasa_cupon,
    usd,
    clp,
    usdclp_obs,
    0,
    es_bono
)

Visualización.

In [23]:
leg_as_dataframe(fixed_rate_mccy_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda_nocional,valor_tasa,tipo_tasa,fecha_fixing_fx,moneda_pago,indice_fx,valor_indice_fx,amortizacion_moneda_pago,interes_moneda_pago
0,1969-01-31,1969-07-31,1969-07-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1969-07-31,CLP,USDOBS,1.00,0.00,"1,500.00"
1,1969-07-31,1970-01-30,1970-01-30,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1970-01-30,CLP,USDOBS,1.00,0.00,"1,500.00"
2,1970-01-30,1970-07-31,1970-07-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1970-07-31,CLP,USDOBS,1.00,0.00,"1,500.00"
3,1970-07-31,1971-01-29,1971-01-29,"100,000.00",0.00,"1,491.67",False,"1,491.67",USD,3.0000%,Lin30360,1971-01-29,CLP,USDOBS,1.00,0.00,"1,491.67"
4,1971-01-29,1971-07-30,1971-07-30,"100,000.00",0.00,"1,508.33",False,"1,508.33",USD,3.0000%,Lin30360,1971-07-30,CLP,USDOBS,1.00,0.00,"1,508.33"
5,1971-07-30,1972-01-31,1972-01-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1972-01-31,CLP,USDOBS,1.00,0.00,"1,500.00"
6,1972-01-31,1972-07-31,1972-07-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1972-07-31,CLP,USDOBS,1.00,0.00,"1,500.00"
7,1972-07-31,1973-01-31,1973-01-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1973-01-31,CLP,USDOBS,1.00,0.00,"1,500.00"
8,1973-01-31,1973-07-31,1973-07-31,"100,000.00",0.00,"1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1973-07-31,CLP,USDOBS,1.00,0.00,"1,500.00"
9,1973-07-31,1974-01-31,1974-01-31,"100,000.00","100,000.00","1,500.00",False,"1,500.00",USD,3.0000%,Lin30360,1974-01-31,CLP,USDOBS,1.00,"100,000.00","1,500.00"


## Construcción Asistida de un `BulletIborLeg`

En este ejemplo se construye un `Leg` con `IborCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `Tenor`: periodicidad de fijación
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular para el calendario de fijaciones
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación
- `unsigned int`: lag de fijación expresado en días
- `InterestRateIndex`: índice de tasa de interés utilizado en cada `IborCashflow`
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `QCCurrency`: moneda del nominal y de los flujos
- `float`: spread aditivo
- `gearing`: spread multiplicativo

**NOTA:** para construir un `Leg` con `IborCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_ibor_leg(...)`.

Vamos a un ejemplo. Primero, se da de alta los parámetros requeridos. 

In [24]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 1, 2026) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
periodicidad_fijacion = qcf.Tenor('3M')
periodo_irregular_fijacion = qcf.StubPeriod.NO

# Se utilizará el mismo calendario para pago y fijaciones
lag_de_fijacion = 2

nominal = 1_000_000.0
amort_es_flujo = True 
moneda = usd
spread = .01
gearing = 1.0

Se define el índice de tasa de interés.

In [25]:
codigo = 'TERMSOFR3M'
lin_act360 = qcf.QCInterestRate(.0, qcf.QCAct360(), qcf.QCLinearWf())
fixing_lag = qcf.Tenor('2d')
tenor = qcf.Tenor('3m')
fixing_calendar = calendario
settlement_calendar = calendario
usd = qcf.QCUSD()
termsofr_3m = qcf.InterestRateIndex(
    codigo,
    lin_act360,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    usd
)

Se da de alta el objeto.

In [26]:
ibor_leg = qcf.LegFactory.build_bullet_ibor_leg(
    rp, 
    fecha_inicio, 
    fecha_final, 
    bus_adj_rule, 
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    lag_pago,
    periodicidad_fijacion, 
    periodo_irregular_fijacion,
    calendario, 
    lag_de_fijacion, 
    termsofr_3m,
    nominal, 
    amort_es_flujo, 
    moneda, 
    spread, 
    gearing
)

Visualización. Notar que los flujos de intereses corresponden al spread de 1%. No están fijados los valores del índice en cada cupón.

In [27]:
leg_as_dataframe(ibor_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa
0,2024-01-31,2024-04-30,2024-01-29,2024-04-30,"1,000,000.00",0.00,"2,500.00",True,"2,500.00",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
1,2024-04-30,2024-07-31,2024-04-26,2024-07-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
2,2024-07-31,2024-10-31,2024-07-29,2024-10-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
3,2024-10-31,2025-01-31,2024-10-29,2025-01-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
4,2025-01-31,2025-04-30,2025-01-29,2025-04-30,"1,000,000.00",0.00,"2,472.22",True,"2,472.22",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
5,2025-04-30,2025-07-31,2025-04-28,2025-07-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
6,2025-07-31,2025-10-31,2025-07-29,2025-10-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
7,2025-10-31,2026-01-30,2025-10-29,2026-01-30,"1,000,000.00","1,000,000.00","2,527.78",True,"1,002,527.78",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360


### Otras Combinaciones de Períodos Irregulares

Las distintas combinaciones de períodos irregulares (de pago y fijación) permiten obtener tablas de desarrollo con muchas características.

#### `SHORTFRONT` con `SHORTFRONT`

In [28]:
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 3, 2026) 
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.SHORTFRONT
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
periodicidad_fijacion = qcf.Tenor('3M')
periodo_irregular_fijacion = qcf.StubPeriod.SHORTFRONT

ibor_leg = qcf.LegFactory.build_bullet_ibor_leg(
    rp, 
    fecha_inicio, 
    fecha_final, 
    bus_adj_rule, 
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    lag_pago,
    periodicidad_fijacion, 
    periodo_irregular_fijacion,
    calendario, 
    lag_de_fijacion, 
    termsofr_3m,
    nominal, 
    amort_es_flujo, 
    moneda, 
    spread, 
    gearing
)

En este caso, las fechas de fijación se sincorinizan con las fechas de inicio de los cupones. Notar además que el primer cupón es de dos meses mientras que el índice es de 3M.

In [29]:
leg_as_dataframe(ibor_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa
0,2024-01-31,2024-03-29,2024-01-29,2024-03-29,"1,000,000.00",0.00,"1,611.11",True,"1,611.11",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
1,2024-03-29,2024-06-28,2024-03-27,2024-06-28,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
2,2024-06-28,2024-09-30,2024-06-26,2024-09-30,"1,000,000.00",0.00,"2,611.11",True,"2,611.11",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
3,2024-09-30,2024-12-31,2024-09-26,2024-12-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
4,2024-12-31,2025-03-31,2024-12-27,2025-03-31,"1,000,000.00",0.00,"2,500.00",True,"2,500.00",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
5,2025-03-31,2025-06-30,2025-03-27,2025-06-30,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
6,2025-06-30,2025-09-30,2025-06-26,2025-09-30,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
7,2025-09-30,2025-12-31,2025-09-26,2025-12-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
8,2025-12-31,2026-03-31,2025-12-29,2026-03-31,"1,000,000.00","1,000,000.00","2,500.00",True,"1,002,500.00",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360


#### `SHORTFRONT` con `SHORTBACK`

In [30]:
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 3, 2026) 
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.SHORTFRONT
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
periodicidad_fijacion = qcf.Tenor('3M')
periodo_irregular_fijacion = qcf.StubPeriod.SHORTBACK

ibor_leg = qcf.LegFactory.build_bullet_ibor_leg(
    rp, 
    fecha_inicio, 
    fecha_final, 
    bus_adj_rule, 
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    lag_pago,
    periodicidad_fijacion, 
    periodo_irregular_fijacion,
    calendario, 
    lag_de_fijacion, 
    termsofr_3m,
    nominal, 
    amort_es_flujo, 
    moneda, 
    spread, 
    gearing
)

En este caso, las fechas de fijación se desfasan respecto a las fechas de inicio de los cupones. Notar que la fecha de fijación de los primeros dos cupones es la misma.

In [31]:
leg_as_dataframe(ibor_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa
0,2024-01-31,2024-03-29,2024-01-29,2024-03-29,"1,000,000.00",0.00,"1,611.11",True,"1,611.11",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
1,2024-03-29,2024-06-28,2024-01-29,2024-06-28,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
2,2024-06-28,2024-09-30,2024-04-26,2024-09-30,"1,000,000.00",0.00,"2,611.11",True,"2,611.11",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
3,2024-09-30,2024-12-31,2024-07-29,2024-12-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
4,2024-12-31,2025-03-31,2024-10-29,2025-03-31,"1,000,000.00",0.00,"2,500.00",True,"2,500.00",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
5,2025-03-31,2025-06-30,2025-01-29,2025-06-30,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
6,2025-06-30,2025-09-30,2025-04-28,2025-09-30,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
7,2025-09-30,2025-12-31,2025-07-29,2025-12-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
8,2025-12-31,2026-03-31,2025-10-29,2026-03-31,"1,000,000.00","1,000,000.00","2,500.00",True,"1,002,500.00",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360


#### `SHORTFRONT` con `LONGBACK`

In [32]:
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 3, 2026) 
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.SHORTFRONT
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
periodicidad_fijacion = qcf.Tenor('3M')
periodo_irregular_fijacion = qcf.StubPeriod.LONGBACK

ibor_leg = qcf.LegFactory.build_bullet_ibor_leg(
    rp, 
    fecha_inicio, 
    fecha_final, 
    bus_adj_rule, 
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    lag_pago,
    periodicidad_fijacion, 
    periodo_irregular_fijacion,
    calendario, 
    lag_de_fijacion, 
    termsofr_3m,
    nominal, 
    amort_es_flujo, 
    moneda, 
    spread, 
    gearing
)

En este caso, las fechas de fijación también se desfasan respecto a las fechas de inicio de los cupones. Ahora las fechas de fijación de los **últimos dos cupones** coinciden.

In [33]:
leg_as_dataframe(ibor_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa
0,2024-01-31,2024-03-29,2024-01-29,2024-03-29,"1,000,000.00",0.00,"1,611.11",True,"1,611.11",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
1,2024-03-29,2024-06-28,2024-01-29,2024-06-28,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
2,2024-06-28,2024-09-30,2024-04-26,2024-09-30,"1,000,000.00",0.00,"2,611.11",True,"2,611.11",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
3,2024-09-30,2024-12-31,2024-07-29,2024-12-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
4,2024-12-31,2025-03-31,2024-10-29,2025-03-31,"1,000,000.00",0.00,"2,500.00",True,"2,500.00",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
5,2025-03-31,2025-06-30,2025-01-29,2025-06-30,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
6,2025-06-30,2025-09-30,2025-04-28,2025-09-30,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
7,2025-09-30,2025-12-31,2025-07-29,2025-12-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360
8,2025-12-31,2026-03-31,2025-10-29,2026-03-31,"1,000,000.00","1,000,000.00","2,500.00",True,"1,002,500.00",USD,TERMSOFR3M,0.0000%,1.0000%,1.00,LinAct360


## Construcción Asistida de un `BulletIborMultiCurrencyLeg`

Se construye un `Leg` con `IborMultiCurrencyCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `Tenor`: periodicidad de fijación
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular para el calendario de fijaciones
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación
- `unsigned int`: lag de fijación expresado en días
- `InterestRateIndex`: índice de tasa de interés utilizado en cada `IborMultiCurrencyCashflow`
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `QCCurrency`: moneda del nominal
- `float`: spread aditivo
- `gearing`: spread multiplicativo
- `QCCurrency`: moneda del nominal
- `QCCurrency`: moneda de pago los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago
- `int`: lag de fijación del FXRateIndex (respecto a settlement date)

Vamos a un ejemplo.

In [34]:
ibor_mccy_leg = qcf.LegFactory.build_bullet_ibor_mccy_leg(
    rp, fecha_inicio, 
    fecha_final, 
    bus_adj_rule, 
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    lag_pago,
    periodicidad_fijacion, 
    periodo_irregular_fijacion,
    calendario, 
    lag_de_fijacion, 
    termsofr_3m,
    nominal, 
    amort_es_flujo, 
    usd, 
    spread, 
    gearing,
    clp, 
    usdclp_obs, 
    0
)

In [35]:
leg_as_dataframe(ibor_mccy_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nocional,amortizacion,interes,amort_es_flujo,flujo,moneda_nocional,codigo_indice_tasa,spread,gearing,valor_tasa,tipo_tasa,fecha_fixing_fx,moneda_pago,codigo_indice_fx,valor_indice_fx,amortizacion_moneda_pago,interes_moneda_pago
0,2024-01-31,2024-03-29,2024-01-29,2024-03-29,"1,000,000.00",0.00,"1,611.11",True,"1,611.11",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2024-03-29,CLP,USDOBS,1.00,0.00,"1,611.11"
1,2024-03-29,2024-06-28,2024-01-29,2024-06-28,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2024-06-28,CLP,USDOBS,1.00,0.00,"2,527.78"
2,2024-06-28,2024-09-30,2024-04-26,2024-09-30,"1,000,000.00",0.00,"2,611.11",True,"2,611.11",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2024-09-30,CLP,USDOBS,1.00,0.00,"2,611.11"
3,2024-09-30,2024-12-31,2024-07-29,2024-12-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2024-12-31,CLP,USDOBS,1.00,0.00,"2,555.56"
4,2024-12-31,2025-03-31,2024-10-29,2025-03-31,"1,000,000.00",0.00,"2,500.00",True,"2,500.00",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2025-03-31,CLP,USDOBS,1.00,0.00,"2,500.00"
5,2025-03-31,2025-06-30,2025-01-29,2025-06-30,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2025-06-30,CLP,USDOBS,1.00,0.00,"2,527.78"
6,2025-06-30,2025-09-30,2025-04-28,2025-09-30,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2025-09-30,CLP,USDOBS,1.00,0.00,"2,555.56"
7,2025-09-30,2025-12-31,2025-07-29,2025-12-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2025-12-31,CLP,USDOBS,1.00,0.00,"2,555.56"
8,2025-12-31,2026-03-31,2025-10-29,2026-03-31,"1,000,000.00","1,000,000.00","2,500.00",True,"1,002,500.00",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2026-03-31,CLP,USDOBS,1.00,"1,000,000.00","2,500.00"


Fijemos el valor del tipo de cambio en los cashflows para ver el efecto en las últimas dos columnas.

In [36]:
for i in range(ibor_mccy_leg.size()):
    ibor_mccy_leg.get_cashflow_at(i).set_fx_rate_index_value(900.0)

In [37]:
leg_as_dataframe(ibor_mccy_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nocional,amortizacion,interes,amort_es_flujo,flujo,moneda_nocional,codigo_indice_tasa,spread,gearing,valor_tasa,tipo_tasa,fecha_fixing_fx,moneda_pago,codigo_indice_fx,valor_indice_fx,amortizacion_moneda_pago,interes_moneda_pago
0,2024-01-31,2024-03-29,2024-01-29,2024-03-29,"1,000,000.00",0.00,"1,611.11",True,"1,611.11",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2024-03-29,CLP,USDOBS,900.00,0.00,"1,450,000.00"
1,2024-03-29,2024-06-28,2024-01-29,2024-06-28,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2024-06-28,CLP,USDOBS,900.00,0.00,"2,275,000.00"
2,2024-06-28,2024-09-30,2024-04-26,2024-09-30,"1,000,000.00",0.00,"2,611.11",True,"2,611.11",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2024-09-30,CLP,USDOBS,900.00,0.00,"2,350,000.00"
3,2024-09-30,2024-12-31,2024-07-29,2024-12-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2024-12-31,CLP,USDOBS,900.00,0.00,"2,300,000.00"
4,2024-12-31,2025-03-31,2024-10-29,2025-03-31,"1,000,000.00",0.00,"2,500.00",True,"2,500.00",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2025-03-31,CLP,USDOBS,900.00,0.00,"2,250,000.00"
5,2025-03-31,2025-06-30,2025-01-29,2025-06-30,"1,000,000.00",0.00,"2,527.78",True,"2,527.78",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2025-06-30,CLP,USDOBS,900.00,0.00,"2,275,000.00"
6,2025-06-30,2025-09-30,2025-04-28,2025-09-30,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2025-09-30,CLP,USDOBS,900.00,0.00,"2,300,000.00"
7,2025-09-30,2025-12-31,2025-07-29,2025-12-31,"1,000,000.00",0.00,"2,555.56",True,"2,555.56",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2025-12-31,CLP,USDOBS,900.00,0.00,"2,300,000.00"
8,2025-12-31,2026-03-31,2025-10-29,2026-03-31,"1,000,000.00","1,000,000.00","2,500.00",True,"1,002,500.00",USD,TERMSOFR3M,1.0000%,1.00,0.0000%,LinAct360,2026-03-31,CLP,USDOBS,900.00,"900,000,000.00","2,250,000.00"


## Construcción Asistida de un `OvernightIndexLeg`

En este ejemplo se construye un `Leg` con `OvernightIndexCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `BusyAdRules`: tipo de ajuste en la fecha de fijación de los valores inicial y final del índice en cada cupón
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación del índice
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `float`: spread multiplicativo
- `QCInterestRate`: representa el tipo de tasa que se usará que se usará para la tasa equivalente
- `string`: nombre del índice overnight a utilizar
- `unsigned int`: número de decimales de la tasa equivalente
- `QCCurrency`: moneda del nocional
- `DatesForEquivalentRate`: enum que indica qué fechas se utilizan en el cálculo de la tasa equivalente (fechas de devengo o de índice)

**NOTA:** para construir un `Leg` con `OvernightIndexCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_overnight_index_leg(...)`.

Vamos al ejemplo. Primeramente, se da de alta los parámetros requeridos

In [38]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 1, 2029) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
index_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('6M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
num_decimales_tasa_eq = 8
lag_pago = 0
nominal = 100_000_000.0
amort_es_flujo = True 
spread = .01
gearing = 1.0
nombre_indice = 'ICPCLP'

Finalmente, se da de alta el objeto.

In [39]:
on_index_leg = qcf.LegFactory.build_bullet_overnight_index_leg(
    rp, 
    fecha_inicio,
    fecha_final, 
    bus_adj_rule, 
    index_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    calendario,
    lag_pago,
    nominal, 
    amort_es_flujo, 
    spread, 
    gearing,
    qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf()),
    nombre_indice,
    num_decimales_tasa_eq,
    clp,
    qcf.DatesForEquivalentRate.ACCRUAL,
)

Se visualiza.

In [40]:
leg_as_dataframe(on_index_leg).style.format(format_dict)

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,valor_indice_inicial,valor_indice_final,valor_tasa_equivalente,tipo_tasa,interes,flujo,spread,gearing
0,2024-01-31,2024-07-31,2024-01-31,2024-07-31,2024-07-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"505,555.56","505,555.56",1.0000%,1.00
1,2024-07-31,2025-01-31,2024-07-31,2025-01-31,2025-01-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"511,111.11","511,111.11",1.0000%,1.00
2,2025-01-31,2025-07-31,2025-01-31,2025-07-31,2025-07-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"502,777.78","502,777.78",1.0000%,1.00
3,2025-07-31,2026-01-30,2025-07-31,2026-01-30,2026-01-30,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"508,333.33","508,333.33",1.0000%,1.00
4,2026-01-30,2026-07-31,2026-01-30,2026-07-31,2026-07-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"505,555.56","505,555.56",1.0000%,1.00
5,2026-07-31,2027-01-29,2026-07-31,2027-01-29,2027-01-29,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"505,555.56","505,555.56",1.0000%,1.00
6,2027-01-29,2027-07-30,2027-01-29,2027-07-30,2027-07-30,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"505,555.56","505,555.56",1.0000%,1.00
7,2027-07-30,2028-01-31,2027-07-30,2028-01-31,2028-01-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"513,888.89","513,888.89",1.0000%,1.00
8,2028-01-31,2028-07-31,2028-01-31,2028-07-31,2028-07-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"505,555.56","505,555.56",1.0000%,1.00
9,2028-07-31,2029-01-31,2028-07-31,2029-01-31,2029-01-31,"100,000,000.00","100,000,000.00",True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"511,111.11","100,511,111.11",1.0000%,1.00


## Construcción Asistida de un `OvernightIndexMultiCurrencyLeg`

En este ejemplo se construye un `Leg` con `OvernightIndexMultiCurrencyCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `BusyAdRules`: tipo de ajuste en la fecha de fijación de los valores inicial y final del índice en cada cupón
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación del índice
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `float`: spread multiplicativo
- `QCInterestRate`: representa el tipo de tasa que se usará que se usará para la tasa equivalente
- `string`: nombre del índice overnight a utilizar
- `unsigned int`: número de decimales de la tasa equivalente
- `QCCurrency`: moneda del nocional
- `DatesForEquivalentRate`: enum que indica qué fechas se utilizan en el cálculo de la tasa equivalente (fechas de devengo o de índice)
- `QCCurrency`: moneda de pago los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago
- `int`: lag de fijación del FXRateIndex (respecto a settlement date)

**NOTA:** para construir un `Leg` con `OvernightIndexMultiCurrencyCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_overnight_index_multi_currency_leg(...)`.

Vamos al ejemplo. Primeramente, se da de alta los parámetros requeridos

In [54]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 1, 2029) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
index_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('6M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
num_decimales_tasa_eq = 8
lag_pago = 0
nominal = 100_000_000.0
amort_es_flujo = True 
spread = .01
gearing = 1.0
nombre_indice = 'ICPCLP'

Finalmente, se da de alta el objeto.

In [70]:
on_index_mccy_leg = qcf.LegFactory.build_bullet_overnight_index_multi_currency_leg(
    rp, 
    fecha_inicio,
    fecha_final, 
    bus_adj_rule, 
    index_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago, 
    calendario, 
    calendario,
    lag_pago,
    nominal, 
    amort_es_flujo, 
    spread, 
    gearing,
    qcf.QCInterestRate(0.0, qcf.QCAct360(), qcf.QCLinearWf()),
    nombre_indice,
    num_decimales_tasa_eq,
    clp,
    qcf.DatesForEquivalentRate.ACCRUAL,
    usd,
    usdclp_obs,
    0,
)

Se visualiza.

In [45]:
leg_as_dataframe(on_index_mccy_leg).style.format(format_dict)

,fecha_inicial_devengo,fecha_final_devengo,fecha_inicial_indice,fecha_final_indice,fecha_pago,nocional,amortizacion,amort_es_flujo,moneda_nocional,nombre_indice,valor_indice_inicial,valor_indice_final,valor_tasa_equivalente,tipo_tasa,interes,flujo,spread,gearing,moneda_pago,indice_fx,fecha_fijacion_indice_fx,valor_indice_fx,interes_moneda_pago,amortizacion_moneda_pago,flujo_moneda_pago
0,2024-01-31,2024-07-31,2024-01-31,2024-07-31,2024-07-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"505,555.56","505,555.56",1.0000%,1.00,USD,USDOBS,2024-07-31,1.00,"505,556.00",0.00,"505,556.00"
1,2024-07-31,2025-01-31,2024-07-31,2025-01-31,2025-01-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"511,111.11","511,111.11",1.0000%,1.00,USD,USDOBS,2025-01-31,1.00,"511,111.00",0.00,"511,111.00"
2,2025-01-31,2025-07-31,2025-01-31,2025-07-31,2025-07-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"502,777.78","502,777.78",1.0000%,1.00,USD,USDOBS,2025-07-31,1.00,"502,778.00",0.00,"502,778.00"
3,2025-07-31,2026-01-30,2025-07-31,2026-01-30,2026-01-30,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"508,333.33","508,333.33",1.0000%,1.00,USD,USDOBS,2026-01-30,1.00,"508,333.00",0.00,"508,333.00"
4,2026-01-30,2026-07-31,2026-01-30,2026-07-31,2026-07-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"505,555.56","505,555.56",1.0000%,1.00,USD,USDOBS,2026-07-31,1.00,"505,556.00",0.00,"505,556.00"
5,2026-07-31,2027-01-29,2026-07-31,2027-01-29,2027-01-29,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"505,555.56","505,555.56",1.0000%,1.00,USD,USDOBS,2027-01-29,1.00,"505,556.00",0.00,"505,556.00"
6,2027-01-29,2027-07-30,2027-01-29,2027-07-30,2027-07-30,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"505,555.56","505,555.56",1.0000%,1.00,USD,USDOBS,2027-07-30,1.00,"505,556.00",0.00,"505,556.00"
7,2027-07-30,2028-01-31,2027-07-30,2028-01-31,2028-01-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"513,888.89","513,888.89",1.0000%,1.00,USD,USDOBS,2028-01-31,1.00,"513,889.00",0.00,"513,889.00"
8,2028-01-31,2028-07-31,2028-01-31,2028-07-31,2028-07-31,"100,000,000.00",0.00,True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"505,555.56","505,555.56",1.0000%,1.00,USD,USDOBS,2028-07-31,1.00,"505,556.00",0.00,"505,556.00"
9,2028-07-31,2029-01-31,2028-07-31,2029-01-31,2029-01-31,"100,000,000.00","100,000,000.00",True,CLP,ICPCLP,1.000000,1.000000,0.000000,LinAct360,"511,111.11","100,511,111.11",1.0000%,1.00,USD,USDOBS,2029-01-31,1.00,"511,111.00","100,000,000.00","100,511,111.00"


## Construcción Asistida de un `IcpClfLeg`
En este ejemplo se construye un `Leg` con `IcpClfCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `gearing`: spread multiplicativo

**NOTA:** para construir un `Leg` con `IcpClfCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_icp_clf_leg(...)`.

Vamos al ejemplo.

Se da de alta los parámetros requeridos

In [47]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 1, 2028)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('6M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True
spread = .01
gearing = 1.0

Se da de alta el objeto.

In [52]:
icp_clf_leg = qcf.LegFactory.build_bullet_icp_clf_leg(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago,
    calendario,
    lag_pago,
    nominal,
    amort_es_flujo,
    spread,
    gearing)

In [55]:
leg_as_dataframe(icp_clf_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,uf_inicial,uf_final,valor_tasa,interes,spread,gearing,tipo_tasa,flujo_en_clp
0,2024-01-31,2024-07-31,2024-07-31,"1,000,000.00",0.00,True,"5,055.56",CLF,"10,000.00","10,000.00","35,000.00","35,000.00",1.0000%,"5,055.56",1.0000%,1.00,LinAct360,"176,944,444.00"
1,2024-07-31,2025-01-31,2025-01-31,"1,000,000.00",0.00,True,"5,111.11",CLF,"10,000.00","10,000.00","35,000.00","35,000.00",1.0000%,"5,111.11",1.0000%,1.00,LinAct360,"178,888,889.00"
2,2025-01-31,2025-07-31,2025-07-31,"1,000,000.00",0.00,True,"5,027.78",CLF,"10,000.00","10,000.00","35,000.00","35,000.00",1.0000%,"5,027.78",1.0000%,1.00,LinAct360,"175,972,222.00"
3,2025-07-31,2026-01-30,2026-01-30,"1,000,000.00",0.00,True,"5,083.33",CLF,"10,000.00","10,000.00","35,000.00","35,000.00",1.0000%,"5,083.33",1.0000%,1.00,LinAct360,"177,916,667.00"
4,2026-01-30,2026-07-31,2026-07-31,"1,000,000.00",0.00,True,"5,055.56",CLF,"10,000.00","10,000.00","35,000.00","35,000.00",1.0000%,"5,055.56",1.0000%,1.00,LinAct360,"176,944,444.00"
5,2026-07-31,2027-01-29,2027-01-29,"1,000,000.00",0.00,True,"5,055.56",CLF,"10,000.00","10,000.00","35,000.00","35,000.00",1.0000%,"5,055.56",1.0000%,1.00,LinAct360,"176,944,444.00"
6,2027-01-29,2027-07-30,2027-07-30,"1,000,000.00",0.00,True,"5,055.56",CLF,"10,000.00","10,000.00","35,000.00","35,000.00",1.0000%,"5,055.56",1.0000%,1.00,LinAct360,"176,944,444.00"
7,2027-07-30,2028-01-31,2028-01-31,"1,000,000.00","1,000,000.00",True,"1,005,138.89",CLF,"10,000.00","10,000.00","35,000.00","35,000.00",1.0000%,"5,138.89",1.0000%,1.00,LinAct360,"35,179,861,111.00"


## Construcción Asistida de un `CompoundedOvernightRateLeg`

En este ejemplo se construye un `Leg` con `CompoundedOvernightRateCashflow2` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación de la tasa overnight
- `QCInterestRateIndex`: índice overnight a utilizar
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `QCCurrency`: moneda del nocional
- `float`: spread aditivo
- `float`: spread multiplicativo
- `QCInterestRate`: representa el tipo de tasa que se usará que se usará para la tasa equivalente
- `unsigned int`: número de decimales de la tasa equivalente
- `unsigned int`: lookback (no implementado)
- `unsigned int`: lockout (no implementado)

**NOTA:** para construir un `Leg` con `CompoundedOvernightRateCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_compounded_overnight_rate_leg_2(...)`.

Vamos al ejemplo. Primeramente, se da de alta los parámetros requeridos

In [63]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 1, 2028)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('6M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True
moneda = usd
spread = .01
gearing = 1.0

Se define el índice.

In [64]:
codigo = 'OISTEST'
lin_act360 = qcf.QCInterestRate(.0, qcf.QCAct360(), qcf.QCLinearWf())
fixing_lag = qcf.Tenor('0d')
tenor = qcf.Tenor('1d')
fixing_calendar = calendario
settlement_calendar = calendario
usd = qcf.QCUSD()
oistest = qcf.InterestRateIndex(
    codigo,
    lin_act360,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    usd
)

Finalmente, se da de alta el objeto.

In [66]:
cor_leg = qcf.LegFactory.build_bullet_compounded_overnight_rate_leg_2(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago,
    calendario,
    lag_pago,
    calendario,
    oistest,
    nominal,
    amort_es_flujo,
    usd,
    spread,
    gearing,
    lin_act360,
    8,
    0,
    0
)

In [67]:
leg_as_dataframe(cor_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,tipo_tasa,valor_tasa,spread,gearing
0,2024-01-31,2024-07-31,2024-07-31,"1,000,000.00",0.00,"5,055.56",True,"5,055.56",USD,OISTEST,LinAct360,0.0000%,1.0000%,1.00
1,2024-07-31,2025-01-31,2025-01-31,"1,000,000.00",0.00,"5,111.11",True,"5,111.11",USD,OISTEST,LinAct360,0.0000%,1.0000%,1.00
2,2025-01-31,2025-07-31,2025-07-31,"1,000,000.00",0.00,"5,027.78",True,"5,027.78",USD,OISTEST,LinAct360,0.0000%,1.0000%,1.00
3,2025-07-31,2026-01-30,2026-01-30,"1,000,000.00",0.00,"5,083.33",True,"5,083.33",USD,OISTEST,LinAct360,0.0000%,1.0000%,1.00
4,2026-01-30,2026-07-31,2026-07-31,"1,000,000.00",0.00,"5,055.56",True,"5,055.56",USD,OISTEST,LinAct360,0.0000%,1.0000%,1.00
5,2026-07-31,2027-01-29,2027-01-29,"1,000,000.00",0.00,"5,055.56",True,"5,055.56",USD,OISTEST,LinAct360,0.0000%,1.0000%,1.00
6,2027-01-29,2027-07-30,2027-07-30,"1,000,000.00",0.00,"5,055.56",True,"5,055.56",USD,OISTEST,LinAct360,0.0000%,1.0000%,1.00
7,2027-07-30,2028-01-31,2028-01-31,"1,000,000.00","1,000,000.00","5,138.89",True,"1,005,138.89",USD,OISTEST,LinAct360,0.0000%,1.0000%,1.00


## Construcción Asistida de un `CompoundedOvernightRateMultiCurrencyLeg`

En este ejemplo se construye un `Leg` con `CompoundedOvernightRateMultiCurrencyCashflow2` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación de la tasa overnight
- `QCInterestRateIndex`: índice overnight a utilizar
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `QCCurrency`: moneda del nocional
- `float`: spread aditivo
- `float`: spread multiplicativo
- `QCInterestRate`: representa el tipo de tasa que se usará que se usará para la tasa equivalente
- `unsigned int`: número de decimales de la tasa equivalente
- `unsigned int`: lookback (no implementado)
- `unsigned int`: lockout (no implementado)
- `QCCurrency`: moneda de pago los flujos
- `FXRateIndex`: índice con el cual se transforma cada flujo a la moneda de pago
- `int`: lag de fijación del FXRateIndex (respecto a settlement date)

**NOTA:** para construir un `Leg` con `CompoundedOvernightRateMultiCurrencyCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_compounded_overnight_rate_multi_currency_leg_2(...)`.

Vamos al ejemplo. Primeramente, se da de alta los parámetros requeridos

In [68]:
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 2024)
fecha_final = qcf.QCDate(31, 1, 2028)
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('6M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True
moneda = usd
spread = .01
gearing = 1.0

Se define el índice.

In [69]:
codigo = 'OISTEST'
lin_act360 = qcf.QCInterestRate(.0, qcf.QCAct360(), qcf.QCLinearWf())
fixing_lag = qcf.Tenor('0d')
tenor = qcf.Tenor('1d')
fixing_calendar = calendario
settlement_calendar = calendario
usd = qcf.QCUSD()
oistest = qcf.InterestRateIndex(
    codigo,
    lin_act360,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    usd
)

Finalmente, se da de alta el objeto.

In [74]:
cor_mccy_leg = qcf.LegFactory.build_bullet_compounded_overnight_rate_mccy_leg_2(
    rp,
    fecha_inicio,
    fecha_final,
    bus_adj_rule,
    periodicidad_pago,
    periodo_irregular_pago,
    calendario,
    lag_pago,
    calendario,
    oistest,
    nominal,
    amort_es_flujo,
    usd,
    spread,
    gearing,
    lin_act360,
    8,
    0,
    0,
    0,
    usd,
    usdclp_obs,
)

In [75]:
leg_as_dataframe(cor_mccy_leg).style.format(format_dict)

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,tipo_tasa,spread,gearing,valor_tasa,moneda_pago,fx_rate_index,fecha_fixing_fx,valor_indice_fx,interes_moneda_pago,amortizacion_moneda_pago,flujo_moneda_pago
0,2024-01-31,2024-07-31,2024-07-31,"1,000,000.00",0.00,"5,055.56",True,"5,055.56",USD,OISTEST,LinAct360,1.0000%,1.00,0.0000%,USD,USDOBS,2024-07-31,1.00,"5,055.56",0.00,"5,055.56"
1,2024-07-31,2025-01-31,2025-01-31,"1,000,000.00",0.00,"5,111.11",True,"5,111.11",USD,OISTEST,LinAct360,1.0000%,1.00,0.0000%,USD,USDOBS,2025-01-31,1.00,"5,111.11",0.00,"5,111.11"
2,2025-01-31,2025-07-31,2025-07-31,"1,000,000.00",0.00,"5,027.78",True,"5,027.78",USD,OISTEST,LinAct360,1.0000%,1.00,0.0000%,USD,USDOBS,2025-07-31,1.00,"5,027.78",0.00,"5,027.78"
3,2025-07-31,2026-01-30,2026-01-30,"1,000,000.00",0.00,"5,083.33",True,"5,083.33",USD,OISTEST,LinAct360,1.0000%,1.00,0.0000%,USD,USDOBS,2026-01-30,1.00,"5,083.33",0.00,"5,083.33"
4,2026-01-30,2026-07-31,2026-07-31,"1,000,000.00",0.00,"5,055.56",True,"5,055.56",USD,OISTEST,LinAct360,1.0000%,1.00,0.0000%,USD,USDOBS,2026-07-31,1.00,"5,055.56",0.00,"5,055.56"
5,2026-07-31,2027-01-29,2027-01-29,"1,000,000.00",0.00,"5,055.56",True,"5,055.56",USD,OISTEST,LinAct360,1.0000%,1.00,0.0000%,USD,USDOBS,2027-01-29,1.00,"5,055.56",0.00,"5,055.56"
6,2027-01-29,2027-07-30,2027-07-30,"1,000,000.00",0.00,"5,055.56",True,"5,055.56",USD,OISTEST,LinAct360,1.0000%,1.00,0.0000%,USD,USDOBS,2027-07-30,1.00,"5,055.56",0.00,"5,055.56"
7,2027-07-30,2028-01-31,2028-01-31,"1,000,000.00","1,000,000.00","5,138.89",True,"1,005,138.89",USD,OISTEST,LinAct360,1.0000%,1.00,0.0000%,USD,USDOBS,2028-01-31,1.00,"5,138.89","1,000,000.00","1,005,138.89"
